In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import scipy as sc
import plotly.graph_objects as go
import sqlite3

In [3]:
end_date = datetime.today()
start_date = end_date - timedelta(days=365)
tickers = ['SPY','BABA', 'AAPL','LLY', 'GIS', 'XOM']
weights = [10,20,20,10,25,15]

In [4]:
# Import data
def getData(tickers:list, start_date:datetime, end_date:datetime):
    
    df = yf.download( tickers,
                    start=start_date,
                    end = end_date
                    )
    df = df['Adj Close']
    returns = df.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

meanReturns, covMatrix = getData(tickers, start_date, end_date)

[*********************100%***********************]  6 of 6 completed


In [5]:
def portfolioPerformance(weights:list, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(
            np.dot(np.array(weights).T,np.dot(covMatrix, weights))
           )*np.sqrt(252)
    return returns, std



In [6]:
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)
returns, std

(20.597688290482512, 23.99410512003802)

# Maximium Sharpe Ratio Portfolio

Highest reward vs risk

what each assets return and variance is going to be into the future and minimise risk while maximising our return. So we have to estimate these parameters, and one way of doing this is using historical stock price data.

In [7]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd

def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [ ]:
result = maxSR(meanReturns, covMatrix)
max_sr, max_weigths = result['fun'], result['x']
print(max_sr, max_weigths)

-1.7341885384020113 [0.         0.04910308 0.57233567 0.08644095 0.         0.29212029]


In [ ]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    """Minimize the portfolio variance by altering the 
     weights/allocation of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [ ]:
min_var_result = minimizeVariance(meanReturns, covMatrix)
min_var, min_var_weights = min_var_result['fun'], min_var_result['x']
print(min_var, min_var_weights)

0.1723342571587694 [0.         0.01400292 0.45633545 0.14706667 0.25569798 0.12689698]


## Creating the Efficient Frontier

In [ ]:
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]


def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """For each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

In [ ]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Read in mean, cov matrix, and other financial information
        Output, Max SR , Min Volatility, efficient frontier """
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns/100, maxSR_returns/100, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns

In [ ]:
print(calculatedResults(meanReturns, covMatrix))

(31.74, 18.3,       allocation
AAPL         0.0
BABA         5.0
GIS         57.0
LLY          9.0
SPY          0.0
XOM         29.0, 21.95, 17.23,       allocation
AAPL         0.0
BABA         1.0
GIS         46.0
LLY         15.0
SPY         26.0
XOM         13.0, [0.1723341465302719, 0.1723577327074908, 0.1724284385939162, 0.17254620712666072, 0.1727109450245184, 0.17292251271650125, 0.17318074279790757, 0.17348542100660141, 0.1738363051681849, 0.1742331184907331, 0.17467555258880993, 0.17516326702517707, 0.17569588450815168, 0.17627298882951054, 0.17689413541124385, 0.17755886547164207, 0.1782667237134094, 0.17904110787820657, 0.18053565753173567, 0.18302508119699232], array([0.2195    , 0.22465263, 0.22980526, 0.23495789, 0.24011053,
       0.24526316, 0.25041579, 0.25556842, 0.26072105, 0.26587368,
       0.27102632, 0.27617895, 0.28133158, 0.28648421, 0.29163684,
       0.29678947, 0.30194211, 0.30709474, 0.31224737, 0.3174    ]))


## Visualising the Efficient Frontier

In [ ]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Return a graph ploting the min vol, max sr and efficient frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )

    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )

    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [ ]:
EF_graph(meanReturns, covMatrix)